In [1]:
!pip freeze | grep scikit-learn

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!python -V

Python 3.11.9


In [9]:
import pickle
import pandas as pd
import numpy as np

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [25]:
print(np.std(y_pred))

6.247488852238703


In [23]:
year=2023
month=3
output_file='q2_output.parquet'

In [14]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [19]:
df_result=df[['ride_id']]

In [20]:
df_result['y_pred']=y_pred

C:\Users\Mohak Kulshreshtha\AppData\Local\Temp\ipykernel_25352\1256420650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['y_pred']=y_pred


In [24]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)